# Homework 2

## 1.1 



In [3]:
def word_prob(good, bad, word, ngood, nbad):
    '''
    compute the probability that a word is spam
    '''
    g = 2 * good.get(word, 0)
    b = bad.get(word, 0)
    if g + b >= 1:
        return max(0.01, min(0.99, min(1.0, b / nbad) / (min(1.0, g / ngood) + min(1.0, b / nbad))))
    return -1


def word_hash(corpus):
    '''
    create a hash for each word in a corpus of the number of times it appears
    '''
    word_hash = {}
    for sample in corpus:
        done = []
        for word in sample:
            # check if word in this sample has already been counted
            if word not in done:
                # check if word already has an entry and if so
                # update the new count
                if word in word_hash:
                    word_hash[word] = sample.count(word) + word_hash[word]
                else:
                    word_hash[word] = sample.count(word)
                done.append(word)
    return word_hash


def unique_counter(unique_list, corpus):
    '''
    add unique words from a corpus to a list
    '''
    for sample in corpus:
        for word in sample:
            # check if word in this sample has already been added
            if word not in unique_list:
                unique_list.append(word)
    return unique_list


def prob_hash(good, bad, ngood, nbad, word_list):
    '''
    find the probability a word means something is spam and put this value in a hash
    '''
    spam_prob = {}
    for word in word_list:
        spam_prob[word] = word_prob(good, bad, word, ngood, nbad)
    return spam_prob


# a corpus has to a be a list of lists which contain strings
spam_corpus = [["i", "am", "spam", "spam", "i", "am"], ["i", "do", "not", "like", "that", "spamiam"]]
ham_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]

# get unique words in a each corpus
word_list = []
word_list = unique_counter(word_list, ham_corpus)
word_list = unique_counter(word_list, spam_corpus)

# get count hashtable for each corpus
spam_hash = word_hash(spam_corpus)
ham_hash = word_hash(ham_corpus)
# get size of each
ngood = len(ham_corpus)
nbad = len(spam_corpus)

# generate a hash of word to probability an email with that word is spam
spam_prob_hash = prob_hash(ham_hash, spam_hash, ngood, nbad, word_list)
print(spam_prob_hash)


{'do': 0.3333333333333333, 'i': 0.5, 'like': 0.3333333333333333, 'green': 0.01, 'eggs': 0.01, 'and': 0.01, 'ham': 0.01, 'am': 0.99, 'spam': 0.99, 'not': 0.99, 'that': 0.99, 'spamiam': 0.99}


## 1.2
This implementation of the spam filter is Baysian 

## 2.1

In [4]:
'''
This module implements the Bayesian network shown in the text, Figure 14.12a.
@student: Noah Madrid
@date: 3-6-20
'''

from probability import BayesNet, enumeration_ask, elimination_ask, gibbs_ask

# Utility variables
T, F = True, False

wet = BayesNet([
    ('Cloudy', '', 0.5),
    ('Sprinkler', 'Cloudy', {T: 0.1, F: 0.50}),
    ('Rain', 'Cloudy', {T: 0.80, F: 0.2}),
    ('WetGrass', 'Sprinkler Rain', {(T, T): 0.99, (T, F): 0.90, (F, T): 0.90, (F, F): 0.00})
    ])

# Compute P(cloudy)
print(enumeration_ask('Cloudy', dict(), wet).show_approx())

# Compute P(Sprinkler | cloudy)
print(enumeration_ask('Sprinkler', dict(Cloudy=T), wet).show_approx())

# Compute P(Cloudy| the sprinkler is running and it’s not raining)
print(enumeration_ask('Cloudy', dict(Sprinkler=T, Raining=F), wet).show_approx())

# Compute P(WetGrass | it’s cloudy, the sprinkler is running and it’s raining)
print(enumeration_ask('WetGrass', dict(Cloudy=T, Sprinkler=T, Raining=T), wet).show_approx())

# Compute P(Cloudy | the grass is not wet)
print(enumeration_ask('Cloudy', dict(WetGrass=F), wet).show_approx())

False: 0.5, True: 0.5
False: 0.9, True: 0.1
False: 0.833, True: 0.167
False: 0.028, True: 0.972
False: 0.639, True: 0.361
